In [103]:
# Gerando o Grafo
import networkx as nx, numpy as np, random as rd


G = nx.Graph()
G.add_nodes_from((0,1,2,3,4))
G.add_edges_from([(0,1),(1,2),(2,3),(3,4),(4,0),(0,2),(0,3),(1,3),(1,4),(2,4)])

#G = nx.random_regular_graph(3,100)

# Criando lista de adjacencia
dic = nx.convert.to_dict_of_dicts(G)
sordic = sorted(dic.keys())
amatrix = {}
for i in sordic:
    amatrix.update({i:dic[i]})

# Taxa de infecção, recuperação e mobilidade
τ = 2.0
γ = 1.0
m = 0.5

# Condições iniciais
initial_infections = [3,]
infected_nodes = initial_infections
tmax = 10
times = [0]
S = [len(amatrix) - len(initial_infections)]
I = [len(initial_infections)]
R = [0]


# Lista de nós recuperados
recovered_nodes = []

# Número de vértices vazios
n = 1
empty_nodes = []
while len(empty_nodes) < n:
    r = rd.choice(list(amatrix))
    if r not in empty_nodes and r not in infected_nodes:
        empty_nodes.append(r)

#print('Nós vazios: ',empty_nodes)
        
# Nós que tem vizinhos vazios
at_mov_nodes = []
for n in empty_nodes:
    viz = list(amatrix[n])
    for v in viz:
        if v not in empty_nodes:
            at_mov_nodes.append(v)
at_mov_nodes = list(set(at_mov_nodes))

#print('at move nodes: ', at_mov_nodes)

# Taxa de mobilidade para os nós que podem se mover
mobility_rate = np.zeros(len(amatrix))
for n in at_mov_nodes:
    viz = list(amatrix[n])
    n_viz = 0
    for vv in viz:
        if vv in empty_nodes:
            n_viz += 1
    mobility_rate[n] = m*n_viz

# lista de suscetíveis
susceptibles_nodes = list(amatrix)
lista = []
for i in susceptibles_nodes:
    if i not in infected_nodes:
        if i not in empty_nodes:
            if i not in recovered_nodes:
                lista.append(i)
susceptibles_nodes = lista        
        
# Nós suscetíveis que tem vizinhos infectados
at_risk_nodes = []
for n in infected_nodes:
    viz = list(amatrix[n])
    for v in viz:
        if v in susceptibles_nodes:
            at_risk_nodes.append(v)
at_risk_nodes = list(set(at_risk_nodes))

# Armazenando as taxas de infecção de cada um dos nós em risco de infecção
infection_rate = np.zeros(len(amatrix))
for n in at_risk_nodes:
    viz = list(amatrix[n])
    n_viz = 0
    for vv in viz:
        if vv in infected_nodes:
            n_viz += 1
    infection_rate[n] = τ*n_viz


# Definindo as taxas de infecção total e recuperação total
total_infection_rate = np.sum(infection_rate)
total_recovery_rate = γ*len(infected_nodes)
total_mobility_rate = np.sum(mobility_rate) # Taxa total de mobilidade
total_rate = total_infection_rate + total_recovery_rate + total_mobility_rate


# Tempo será selecionado desta forma
time = -np.log(rd.random())/total_rate


# Armazenando os nós recuperados e quantos que se moveram para fins de plot.
moved = [0]
mob_count = 0


print(f'Nós suscetíveis: {susceptibles_nodes}')
print(f'Nós infectados: {infected_nodes}')
print(f'Nós recuperados: {recovered_nodes}')
print(f'Nós vazios: {empty_nodes}')



# Início do algoritmo
import random as rd
while time < tmax and total_rate > 0:

    r = rd.uniform(0,1)
    r_rec = total_recovery_rate/total_rate
    r_inf = total_infection_rate/total_rate
    r_mob = total_mobility_rate/total_rate
    
    if r < r_rec:
        
        u = rd.choice(infected_nodes)

        infected_nodes.pop(infected_nodes.index(u))
        recovered_nodes.append(u)

        uviz = list(amatrix[u])
        for v in uviz:
            if v in at_risk_nodes:
                infection_rate[v] -= τ
                
        print(f'\nRecuperação.')
        print(f'{u} se recuperou.')
        print(f'Taxa de infecção:{infection_rate}.')
        print(f'Nó suscetível: ',susceptibles_nodes)

    elif r_rec < r <= (r_rec + r_inf):
        
        p = infection_rate/total_infection_rate
        p_list_inf = []
        for i in at_risk_nodes:
            p_list_inf.append(p[i])
        u = rd.choices(at_risk_nodes, weights=p_list_inf)[0]

        susceptibles_nodes.pop(susceptibles_nodes.index(u))
        at_risk_nodes.pop(at_risk_nodes.index(u))
        infected_nodes.append(u)
        infection_rate[u] = 0

        uviz = list(amatrix[u])
        for v in uviz:
            if v in susceptibles_nodes:
                if v not in at_risk_nodes:
                    at_risk_nodes.append(v)
                infection_rate[v] += τ

        print('\nInfecção')
        print(f'{u} se infecta')
        print(f'Taxa de infecção: {infection_rate}')
        print(f'Nós infectados: {infected_nodes}')
        print(f'Nó suscetível: ',susceptibles_nodes)


    elif (r_rec + r_inf) < r <= 1:
        
        
        mob_count += 1

        p = mobility_rate/total_mobility_rate
        p_list = []
        for i in at_mov_nodes:
            p_list.append(p[i])
        u = rd.choices(at_mov_nodes, weights=p_list)[0]

        uviz = list(amatrix[u])
        emp_list = []
        for i in uviz:
            if i in empty_nodes:
                emp_list.append(i)
        e = rd.choices(emp_list)[0]

        empty_nodes.pop(empty_nodes.index(e))
        empty_nodes.append(u)


        u_viz = list(amatrix[u])
        for v in u_viz:
            if i not in empty_nodes:
                at_mov_nodes.append(v)
                mobility_rate[v] += m    

        e_viz = list(amatrix[e])
        for v in e_viz:
            if v in at_mov_nodes:
                at_mov_nodes.pop(at_mov_nodes.index(v))
                mobility_rate[v] -= m

        at_mov_nodes = list(set(at_mov_nodes))

        if u in infected_nodes:

            u_compartment = 'I'
        
            eviz = list(amatrix[e])
            for v in eviz:
                if v in susceptibles_nodes:
                    if v not in at_risk_nodes:
                        at_risk_nodes.append(v)
                    infection_rate[v] += τ
                    
            uviz = list(amatrix[u])
            for v in uviz:
                if v in at_risk_nodes:
                    infection_rate[v] -= τ
            
            infected_nodes.pop(infected_nodes.index(u))
            infected_nodes.append(e)

        elif u in susceptibles_nodes:
            
            u_compartment = 'S'
            
            eviz = list(amatrix[e])
            for v in eviz:
                if v in infected_nodes:
                    infection_rate[e] += τ
        
            # Removo u da lista de suscetíveis e adiciono 'e' nesta lista
            susceptibles_nodes.pop(susceptibles_nodes.index(u))
            susceptibles_nodes.append(e)
            if u in at_risk_nodes:
                at_risk_nodes.pop(at_risk_nodes.index(u))

                # Verificar se estará em risco novamente ou não .
                e_viz = list(amatrix[e])
                at_risc = False
                for v in e_viz:
                    if v in infected_nodes:
                        infection_rate[e] += τ
                        at_risc = True

                if at_risc == True:
                    at_risk_nodes.append(e)

            
        # Se 'u' é um recuperado
        elif u in recovered_nodes:
            
            # Removo u da lista de recuperados e adiciono 'e' nesta lista
            recovered_nodes.pop(recovered_nodes.index(u))
            recovered_nodes.append(e)
            
            u_compartment = 'R'

        print('\nMovimentação')
        print(f'Compartimento: {u_compartment}')
        print(f'{u} ocupado troca com {e} vazio')
        print(f'Nó suscetível: ',susceptibles_nodes)


    # Armazenando dados para fins de plot
    S.append(len(G.nodes) - len(infected_nodes) - len(recovered_nodes))
    
    I.append(len(infected_nodes))
    
    R.append(len(recovered_nodes))
    
    moved.append(mob_count)
    
    times.append(time)
    
    # Atualizando as taxas e o tempo
    total_recovery_rate = γ*len(infected_nodes)
    total_infection_rate = np.sum(infection_rate)
    total_mobility_rate = m*len(G.nodes) # Taxa total de mobilidade
    total_rate = total_infection_rate + total_recovery_rate + total_mobility_rate
    
    time = time - np.log(rd.random())/total_rate

import matplotlib.pyplot as plt

Fig, ax = plt.subplots(nrows=2, figsize=(10,6))
ax[0].plot(times, S, label='S', color='green')
ax[0].plot(times, I, label='I', color='red')
ax[0].plot(times, R, label='R', color='blue')
ax[1].plot(times, moved, label='M', color='k')

ax[0].set_ylabel('Nodes')
ax[1].set_ylabel('Number of moves')
ax[1].set_xlabel('Time')
ax[0].legend()
ax[1].legend()

Nós suscetíveis: [1, 2, 4]
Nós infectados: [3]
Nós recuperados: []
Nós vazios: [0]

Movimentação
Compartimento: S
1 ocupado troca com 0 vazio
Nó suscetível:  [2, 4, 0]

Infecção
4 se infecta
Taxa de infecção: [6. 2. 4. 0. 0.]
Nós infectados: [3, 4]
Nó suscetível:  [2, 0]

Movimentação
Compartimento: I
4 ocupado troca com 1 vazio
Nó suscetível:  [2, 0]

Infecção
0 se infecta
Taxa de infecção: [0. 2. 6. 0. 0.]
Nós infectados: [3, 1, 0]
Nó suscetível:  [2]

Infecção
2 se infecta
Taxa de infecção: [0. 2. 0. 0. 0.]
Nós infectados: [3, 1, 0, 2]
Nó suscetível:  []


IndexError: list index out of range

In [22]:
susceptibles_nodes = [0,1,2,3,4]
infected_nodes = [3]
empty_nodes = [0]

In [79]:
susceptibles_nodes = [0,1,2,3,4]
infected_nodes = [3]
empty_nodes = [0]
recovered_nodes = []
lista = []
for i in susceptibles_nodes:
    if i not in infected_nodes:
        if i not in empty_nodes:
            if i not in recovered_nodes:
                lista.append(i)
lista

[1, 2, 4]